In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scikitplot
import seaborn as sns

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [5]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import (
    Dropout,
    BatchNormalization,
    LeakyReLU,
    Activation,
    ReLU,
    Input
)
from tensorflow.keras.callbacks import (
    Callback,
    EarlyStopping,
    ReduceLROnPlateau,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.preprocessing import image
import cv2

from keras.utils import np_utils

In [6]:
data_df = pd.read_csv("../input/fer-data-upload/fer_data.csv")

In [7]:
emotion_nos = np.sort(data_df["emotion"].unique())
emotion_names = [
    "anger",
    "disgust",
    "fear",
    "happiness",
    "sadness",
    "surprise",
    "neutral",
]
fer_dictionary = dict(zip(emotion_nos, emotion_names))
img_dim = 48

In [8]:
img_array = data_df.pixels.apply(
    lambda x: np.array(x.split(" "))
    .reshape(img_dim, img_dim, 1)
    .astype("float32")
)
img_array = np.stack(img_array, axis=0)
img_array.shape

(35887, 48, 48, 1)

In [9]:
labenc = LabelEncoder()
img_labels = labenc.fit_transform(data_df.emotion)
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(35887, 7)

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(
    img_array,
    img_labels,
    shuffle=True,
    stratify=img_labels,
    test_size=0.1,
    random_state=42,
)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((32298, 48, 48, 1), (3589, 48, 48, 1), (32298, 7), (3589, 7))

In [11]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]
feat_dim = (img_width, img_height, 1)

# normalize pixel values to be in [0, 1] range
X_train = X_train / 255.0
X_valid = X_valid / 255.0

In [12]:
def build_net(optim):
    inpx = Input(shape=(48, 48, 1))
    layer_1 = Conv2D(
                filters=64,
                kernel_size=(5, 5),
                input_shape=(img_width, img_height, img_depth),
                activation="relu",
                padding="same",
                kernel_initializer="he_normal",
                name="conv2d_1",
        )(inpx)
    layer_2 = BatchNormalization(name="batchnorm_1")(layer_1)
    layer_3 = Conv2D(
            filters=64,
            kernel_size=(5, 5),
            activation="relu",
            padding="same",
            kernel_initializer="he_normal",
            name="conv2d_2",
        )(layer_2)
    
    layer_4 = BatchNormalization(name="batchnorm_2")(layer_3)
    layer_5 = MaxPooling2D(pool_size=(2, 2), name="maxpool2d_1")(layer_4)
    layer_6 = Dropout(rate=0.4, name="dropout_1")(layer_5)

    layer_7 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_normal",
            name="conv2d_3",
        )(layer_6)
    
    layer_8 = BatchNormalization(name="batchnorm_3")(layer_7)

    layer_9 = Conv2D(
            filters=128,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_normal",
            name="conv2d_4",
        )(layer_8)
    
    layer_10 = BatchNormalization(name="batchnorm_4")(layer_9)
    layer_11 = MaxPooling2D(pool_size=(2, 2), name="maxpool2d_2")(layer_10)
    layer_12 = Dropout(0.4, name="dropout_2")(layer_11)

    layer_13 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_normal",
            name="conv2d_5",
        )(layer_12)
    
    layer_14 = BatchNormalization(name="batchnorm_5")(layer_13)
    
    layer_15 = Conv2D(
            filters=256,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_normal",
            name="conv2d_6",
        )(layer_14)
    
    layer_16 = BatchNormalization(name="batchnorm_6")(layer_15)
    layer_17 = MaxPooling2D(pool_size=(2, 2), name="maxpool2d_3")(layer_16)
    layer_18 = Dropout(rate=0.5, name="dropout_3")(layer_17)
    layer_19 = Flatten(name="flatten")(layer_18)
    
    layer_20 = Dense(
            units=128,
            activation="relu",
            kernel_initializer="he_normal",
            name="dense_1",
        )(layer_19)
    
    layer_21 = BatchNormalization(name="batchnorm_7")(layer_20)
    layer_22 = Dropout(rate=0.6, name="dropout_4")(layer_21)

    layer_23 = Dense(units=num_classes, activation="softmax", name="out_layer")(layer_22)
    
    net = Model([inpx], layer_23)
    
    net.compile(
        loss="categorical_crossentropy", optimizer=optim, metrics=["accuracy"]
    )

    net.summary()

    return net

In [13]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor="val_accuracy",
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [14]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

In [15]:
batch_size = 32  # batch size of 32 performs the best.
epochs = 100

model = build_net(optimizers.Adam(0.001))
history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    callbacks=callbacks,
    use_multiprocessing=True
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
batchnorm_1 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batchnorm_2 (BatchNormalizat (None, 48, 48, 64)        256       
_________________________________________________________________
maxpool2d_1 (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0     

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1009/1009 [==============================] - 38s 28ms/step - loss: 2.0528 - accuracy: 0.2343 - val_loss: 1.6539 - val_accuracy: 0.3566
Epoch 2/100
1009/1009 [==============================] - 27s 27ms/step - loss: 1.7001 - accuracy: 0.3225 - val_loss: 1.5230 - val_accuracy: 0.4193
Epoch 3/100
1009/1009 [==============================] - 28s 27ms/step - loss: 1.5497 - accuracy: 0.3964 - val_loss: 1.3174 - val_accuracy: 0.4918
Epoch 4/100
1009/1009 [==============================] - 26s 26ms/step - loss: 1.4303 - accuracy: 0.4469 - val_loss: 1.2580 - val_accuracy: 0.5174
Epoch 5/100
1009/1009 [==============================] - 26s 26ms/step - loss: 1.3497 - accuracy: 0.4850 - val_loss: 1.2975 - val_accuracy: 0.4935
Epoch 6/100
1009/1009 [==============================] - 27s 27ms/step - loss: 1.3044 - accuracy: 0.5039 - val_loss: 1.1533 - val_accuracy: 0.5614
Epoch 7/100
1009/1009 [==============================] - 27s 27ms/step - loss: 1.2653 - accuracy: 0.5202 - val_loss: 1.1597 - val_

In [16]:
model.save("EmotionDetectionModelFunctional.h5")